In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import yfinance as yf
import os  # To handle file paths
import csv  # To parse the NASDAQ listed file
import sys  # For exit
import random  # For random sampling (if needed)
import json  # For saving the JSON file

# Step 1: Fetch data from Yahoo Finance
def parse_nasdaq_listed(filename):
    symbols = []

    with open(filename, 'r') as file:
        reader = csv.reader(file, delimiter='|')  # Pipe-delimited file

        # Skip the header row
        next(reader)

        # Process each row
        for row in reader:
            # Stop when reaching the footer (which starts with "File Creation Time")
            if 'File Creation Time' in row[0]:
                break

            # The first column contains the stock symbol
            symbol = row[0]
            if symbol:  # Ensure it's not empty
                symbols.append(symbol)

    return symbols

def fetch_stock_data(ticker, period='5y'):
    try:
        # Try to fetch data for the specified period
        stock_data = yf.download(ticker, period=period, interval='1wk')  # Weekly data
        if stock_data.empty:
            print(f"No data fetched for {ticker} with period '{period}'. Trying maximum available data.")
            # Try fetching maximum available data
            stock_data = yf.download(ticker, period='max', interval='1wk')
            if stock_data.empty:
                print(f"No data available for {ticker}. Skipping.")
                return None
        stock_data.reset_index(inplace=True)
        # Ensure 'Date' column exists
        if 'Date' not in stock_data.columns:
            stock_data.rename(columns={'index': 'Date'}, inplace=True)
        stock_data['Ticker'] = ticker
        # Select necessary columns
        stock_data = stock_data[['Date', 'Ticker', 'Close']]
        # Drop rows with NaN values
        stock_data.dropna(inplace=True)
        # Sort data
        stock_data.sort_values(by=['Date'], inplace=True)
        return stock_data
    except yf.shared.YFInvalidPeriodError as e:
        print(f"Invalid period error for {ticker}: {e}")
        return None
    except Exception as e:
        print(f"An error occurred while fetching data for {ticker}: {e}")
        return None

# Step 2: Prepare data
def prepare_data(ticker_data, lookback=4):
    features = ticker_data[['Close']].values
    if len(features) < lookback + 1:
        print(f"Not enough data for {ticker_data['Ticker'].iloc[0]} after considering lookback. Skipping this ticker.")
        return None, None
    X = []
    y = []
    for i in range(lookback, len(features)):
        X.append(features[i - lookback:i])
        y.append(features[i][0])
    return np.array(X), np.array(y)

# Step 3: Train LSTM model
def train_lstm_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, input_shape):
    # Build the LSTM model
    model = tf.keras.models.Sequential([
        tf.keras.layers.LSTM(64, return_sequences=True, input_shape=input_shape),
        tf.keras.layers.LSTM(32),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam', loss='mse')

    # Train the model
    model.fit(
        X_train_scaled, y_train_scaled,
        epochs=20, batch_size=32,
        validation_data=(X_val_scaled, y_val_scaled),
        verbose=0  # Suppress training output
    )

    return model

# Step 4: Predict future stock prices
def predict_future_prices(model, features, scaler, future_weeks, lookback=4):
    # Ensure there is enough data
    if len(features) < lookback:
        print(f"Not enough data to make predictions. Skipping.")
        return None
    # Get the last 'lookback' data points
    input_sequence = features[-lookback:]

    # Scale the input sequence
    input_sequence_scaled = scaler.transform(input_sequence)

    predictions = []

    # Predict future prices
    for _ in range(future_weeks):
        input_sequence_scaled = input_sequence_scaled.reshape(1, lookback, 1)
        predicted_price_scaled = model.predict(input_sequence_scaled)
        predicted_price = scaler.inverse_transform(predicted_price_scaled)[0][0]
        # Convert predicted_price to native float
        predicted_price = float(predicted_price)
        predictions.append(predicted_price)

        # Update the input sequence
        input_sequence = np.append(input_sequence[1:], [[predicted_price]], axis=0)
        input_sequence_scaled = scaler.transform(input_sequence)

    return predictions

# Step 5: Calculate price growth and organize data
def calculate_price_growth(last_close_price, predictions, ticker):
    # Ensure last_close_price is a native float
    last_close_price = float(last_close_price)

    prediction_dict = {}
    for week_ahead, predicted_price in enumerate(predictions, start=1):
        # predicted_price is already a native float
        growth = predicted_price - last_close_price
        week_key = f"week{week_ahead}Prediction"
        prediction_dict[week_key] = {
            "Predicted Price": predicted_price,
            "Price Growth": growth
        }
    return {ticker: prediction_dict}

# Step 6: Plot historical data and predictions, and save as JPEG
def plot_historical_and_predictions(ticker_data, predictions, future_weeks):
    ticker = ticker_data['Ticker'].iloc[0]
    # Get the last 1 year of historical data, or all data if less than 1 year
    last_one_year_date = ticker_data['Date'].max() - pd.DateOffset(years=1)
    if ticker_data['Date'].min() > last_one_year_date:
        ticker_data_last_one_year = ticker_data
    else:
        ticker_data_last_one_year = ticker_data[ticker_data['Date'] >= last_one_year_date]

    # Prepare data for plotting
    dates = list(ticker_data_last_one_year['Date'])
    prices = list(ticker_data_last_one_year['Close'])

    # Prepare future dates for predictions
    last_date = ticker_data['Date'].max()
    future_dates = [last_date + pd.DateOffset(weeks=1 * i) for i in range(1, future_weeks + 1)]

    # Plot historical prices
    plt.figure(figsize=(12, 6))
    plt.plot(dates, prices, label='Historical Prices', color='blue')

    # Plot predicted prices
    if predictions:
        plt.plot(future_dates, predictions, marker='o', linestyle='--', label='Predicted Prices', color='red')

    plt.title(f'Historical and Predicted Prices for {ticker}')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)

    # Save the plot as a JPEG file
    filename = f"{ticker}_historical_and_predicted_prices.jpg"
    plt.savefig(filename, format='jpg')
    plt.close()  # Close the figure to free memory

    print(f"Plot saved for {ticker} as {filename}")

# Main execution
if __name__ == "__main__":
    # Ensure 'nasdaqlisted.txt' exists
    if not os.path.exists('nasdaqlisted.txt'):
        print("File 'nasdaqlisted.txt' not found. Please ensure it exists in the current directory.")
        sys.exit(1)

    # Parse tickers from 'nasdaqlisted.txt'
    all_tickers = parse_nasdaq_listed('nasdaqlisted.txt')

    # Limit the number of tickers to prevent overloading
    # For example, process the first 50 tickers
    tickers_to_process = all_tickers[:10]

    period = '5y'  # Period of historical data to fetch (5 years)
    future_weeks = 4  # Number of weeks to predict into the future
    lookback = 4  # Number of weeks to look back for predictions

    # Dictionary to store price growth data for all tickers
    all_predictions = {}

    for ticker in tickers_to_process:
        print(f"\nProcessing ticker: {ticker}")

        # Step 1: Fetch stock data
        ticker_data = fetch_stock_data(ticker, period)
        if ticker_data is None:
            continue

        # Step 2: Prepare the data
        X, y = prepare_data(ticker_data, lookback)
        if X is None or y is None:
            continue

        # Split data into training, validation, and test sets
        total_samples = len(X)
        if total_samples < 10:
            print(f"Not enough data to split into training, validation, and test sets for {ticker}. Skipping.")
            continue
        train_size = int(total_samples * 0.6)
        val_size = int(total_samples * 0.2)

        X_train, X_val, X_test = X[:train_size], X[train_size:train_size + val_size], X[train_size + val_size:]
        y_train, y_val, y_test = y[:train_size], y[train_size:train_size + val_size], y[train_size + val_size:]

        # Scale data
        scaler = MinMaxScaler()
        X_train_reshaped = X_train.reshape(-1, 1)
        y_train_reshaped = y_train.reshape(-1, 1)
        scaler.fit(np.vstack((X_train_reshaped, y_train_reshaped)))

        X_train_scaled = scaler.transform(X_train_reshaped).reshape(X_train.shape)
        y_train_scaled = scaler.transform(y_train_reshaped)

        X_val_reshaped = X_val.reshape(-1, 1)
        y_val_reshaped = y_val.reshape(-1, 1)
        X_val_scaled = scaler.transform(X_val_reshaped).reshape(X_val.shape)
        y_val_scaled = scaler.transform(y_val_reshaped)

        # Step 3: Train the model
        input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
        model = train_lstm_model(X_train_scaled, y_train_scaled, X_val_scaled, y_val_scaled, input_shape)

        # Step 4: Predict future prices
        features = ticker_data[['Close']].values
        predictions = predict_future_prices(model, features, scaler, future_weeks, lookback)
        if predictions is None:
            continue

        # Step 5: Calculate price growth and organize data
        last_close_price = ticker_data['Close'].iloc[-1]
        ticker_predictions = calculate_price_growth(last_close_price, predictions, ticker)

        # Collect predictions
        all_predictions.update(ticker_predictions)

        # Step 6: Plot the historical data and predictions, and save the plot
        plot_historical_and_predictions(ticker_data, predictions, future_weeks)

    # Save the combined predictions to a JSON file
    output_filename = 'predicted_growth_data.json'
    with open(output_filename, 'w') as json_file:
        json.dump(all_predictions, json_file, indent=4)
    print(f"\nPredicted growth data has been saved to '{output_filename}'")


[*********************100%***********************]  1 of 1 completed


Processing ticker: AACG



<ipython-input-10-28b043bd1c4f>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.dropna(inplace=True)
<ipython-input-10-28b043bd1c4f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.sort_values(by=['Date'], inplace=True)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Plot saved for AACG as AACG_historical_and_predicted_prices.jpg

Processing ticker: AADI


[*********************100%***********************]  1 of 1 completed
<ipython-input-10-28b043bd1c4f>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.dropna(inplace=True)
<ipython-input-10-28b043bd1c4f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.sort_values(by=['Date'], inplace=True)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Plot saved for AAPB as AAPB_historical_and_predicted_prices.jpg

Processing ticker: AAPD


[*********************100%***********************]  1 of 1 completed
<ipython-input-10-28b043bd1c4f>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.dropna(inplace=True)
<ipython-input-10-28b043bd1c4f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.sort_values(by=['Date'], inplace=True)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Plot saved for AAPD as AAPD_historical_and_predicted_prices.jpg

Processing ticker: AAPL


[*********************100%***********************]  1 of 1 completed
<ipython-input-10-28b043bd1c4f>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.dropna(inplace=True)
<ipython-input-10-28b043bd1c4f>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.sort_values(by=['Date'], inplace=True)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Plot saved for AAPL as AAPL_historical_and_predicted_prices.jpg

Predicted growth data has been saved to 'predicted_growth_data.json'
